# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negat

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables": {"a": [0.0, 1000.0], "b": [-1.0, 1.0]}, "constraints": {"e": ["LESS_THAN", 2.0], "f": ["GREATER_THAN", 0.0]}, "objectives": {"c": "MAXIMIZE", "d": "MINIMIZE"}, "constants": {"g": 1234}, "linked_variables": {}}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,584.436124,0.173106,1234,584.609230,584.263017,1169.218460,1168.526034
6,38.245964,-0.773880,1234,37.472083,39.019844,74.944166,78.039688
7,351.002650,0.327409,1234,351.330059,350.675241,702.660117,701.350483
8,326.177828,0.713600,1234,326.891428,325.464228,653.782857,650.928456
9,991.057863,0.620803,1234,991.678666,990.437060,1983.357331,1980.874121
10,630.204360,0.519800,1234,630.724160,629.684560,1261.448320,1259.369119
11,881.823010,-0.281411,1234,881.541600,882.104421,1763.083199,1764.208842
12,323.956591,0.128965,1234,324.085555,323.827626,648.171111,647.655253
13,168.423032,0.481839,1234,168.904871,167.941193,337.809741,335.882385
14,14.960294,-0.393234,1234,14.567059,15.353528,29.134119,30.707056


In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-584.609230,584.263017
6,-37.472083,39.019844
7,-351.330059,350.675241
8,-326.891428,325.464228
9,-991.678666,990.437060
10,-630.724160,629.684560
11,-881.541600,882.104421
12,-324.085555,323.827626
13,-168.904871,167.941193
14,-14.567059,15.353528


In [13]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-584.609230,584.263017
6,-37.472083,39.019844
7,-351.330059,350.675241
8,-326.891428,325.464228
9,-991.678666,990.437060
10,-630.724160,629.684560
11,-881.541600,882.104421
12,-324.085555,323.827626
13,-168.904871,167.941193
14,-14.567059,15.353528


In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-584.60922978,  584.26301724],
       [ -37.47208321,   39.01984413],
       [-351.33005858,  350.67524129],
       [-326.89142842,  325.46422795],
       [-991.67866573,  990.43706036],
       [-630.72416013,  629.68455957],
       [-881.54159954,  882.10442124],
       [-324.08555533,  323.82762627],
       [-168.90487065,  167.94119256],
       [ -14.56705928,   15.35352813]])

In [15]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,1167.218460,-1168.526034
6,72.944166,-78.039688
7,700.660117,-701.350483
8,651.782857,-650.928456
9,1981.357331,-1980.874121
10,1259.448320,-1259.369119
11,1761.083199,-1764.208842
12,646.171111,-647.655253
13,335.809741,-335.882385
14,27.134119,-30.707056


In [16]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
